In [ ]:
import os
import jCMIP as jc
Clist = jc.readList(('CMIP6listnew'))

from netCDF4 import Dataset
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import cftime

#### Add transport of overlapping RAPID period of all models to section26data file

##### Including Bering Strait outflow

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

for i in range (len(totmod)):
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'r')
    v    = ncid.variables['vo'][:,:,0,:]
    dx   = ncid.variables['dx'][0,:]
    dz   = ncid.variables['dz'][:,0,:]
    lev  = ncid.variables['lev'][:]
    lon  = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    ncid.close()
    
    time10 = time[-129::]          #time[-129] is 2004-04-16 00:00:00
    v1 = v[-129::,:,:]
    #Compute transport for every v-point and last 15 years
    tp10 = np.zeros(np.shape(v1))
    for l in range (len(time10)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp10[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6
            
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'a')
    ncid.createDimension('time10',len(time10))
    ncid.createVariable('tp10','f8',('time10','lev','lon',))
    ncid.variables['tp10'][:,:,:]       = tp10
    ncid.close()

##### Excluding Bering Strait outflow

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

for i in range (len(totmod)):
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid    = Dataset(outfile,'r')
    v       = ncid.variables['vo'][:,:,0,:]
    dx      = ncid.variables['dx'][0,:]
    dz      = ncid.variables['dz'][:,0,:]
    lev     = ncid.variables['lev'][:]
    lon     = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    tp10    = ncid.variables['tp10'][:,:,:]
    ncid.close()
    
    dzdx = np.zeros(np.shape(dz))
    for j in range (len(dz)):
        for k in range (len(dx)):
            dzdx[j,k] = dz[j,k]*dx[k]
    
    #Exclude unmasked grid cells in MRI-ESM2
    if totmod[i] == 'MRI-ESM2-0':
        dzdx[-3,14] = 0
        dzdx[-4,31] = 0
        dzdx[-5,40] = 0
    
    dzdxsum  = np.nansum(np.nansum(dzdx,axis=0),axis=0)
    tpsum    = np.nansum(np.nansum(tp10,axis=1),axis=1)
    
    time10 = time[-129::]          #time[-129] is 2004-04-16 00:00:00
    v1 = v[-129::,:,:]
    #Compute transport for every v-point and last 15 years
    tp10_BS = np.zeros(np.shape(v1))
    for l in range (len(time10)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp10_BS[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6 - (tpsum[l]*dzdx[j,k]/dzdxsum)   #Subtract Bering Strait outflow
                
            
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'a')
    ncid.createVariable('tp10_BS','f8',('time10','lev','lon',))
    ncid.variables['tp10_BS'][:,:,:]       = tp10_BS
    ncid.close()

#### Add transport until 2100 of all models to section26data file

##### Including Bering Strait outflow

In [ ]:
#CNRM-ESM2-1 (r2i1p1f2) has no ssp585
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'ssp585'

for i in range (len(totmod)):
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'r')
    v    = ncid.variables['vo'][:,:,0,:]
    dx   = ncid.variables['dx'][0,:]
    dz   = ncid.variables['dz'][:,0,:]
    lev  = ncid.variables['lev'][:]
    lon  = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    ncid.close()
    
    time85 = time[0:85*12]
    v1 = v[0:85*12,:,:]
    #Compute transport for every v-point and future 85 years
    tp85 = np.zeros(np.shape(v1))
    for l in range (len(time85)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp85[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6
            
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'a')
    ncid.createDimension('time85',len(time85))
    ncid.createVariable('tp85','f8',('time85','lev','lon',))
    ncid.variables['tp85'][:,:,:]       = tp85
    ncid.close()

##### Excluding Bering Strait outflow

In [ ]:
#CNRM-ESM2-1 (r2i1p1f2) has no ssp585
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'ssp585'

for i in range (len(totmod)):
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid    = Dataset(outfile,'r')
    v       = ncid.variables['vo'][:,:,0,:]
    dx      = ncid.variables['dx'][0,:]
    dz      = ncid.variables['dz'][:,0,:]
    lev     = ncid.variables['lev'][:]
    lon     = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    tp85    = ncid.variables['tp85'][:,:,:]
    ncid.close()
    
    dzdx = np.zeros(np.shape(dz))
    for j in range (len(dz)):
        for k in range (len(dx)):
            dzdx[j,k] = dz[j,k]*dx[k]

    #Exclude unmasked grid cells in MRI-ESM2            
    if totmod[i] == 'MRI-ESM2-0':
    dzdx[-3,14] = 0
    dzdx[-4,31] = 0
    dzdx[-5,40] = 0
    
    dzdxsum  = np.nansum(np.nansum(dzdx,axis=0),axis=0)
    tpsum    = np.nansum(np.nansum(tp85,axis=1),axis=1)
    
    time85 = time[0:85*12]
    v1 = v[0:85*12,:,:]
    #Compute transport for every v-point and future 85 years
    tp85_BS = np.zeros(np.shape(v1))
    for l in range (len(time85)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp85_BS[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6 - (tpsum[l]*dzdx[j,k]/dzdxsum)   #Subtract Bering Strait outflow
            
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'a')
    ncid.createVariable('tp85_BS','f8',('time85','lev','lon',))
    ncid.variables['tp85_BS'][:,:,:]       = tp85_BS
    ncid.close()

#### Transport since 1950 (for lagged regression)

Including Bering Strait outflow

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

for i in range (len(totmod)):
    print('File ' + str(i+1))
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid = Dataset(outfile,'r')
    v    = ncid.variables['vo'][:,:,0,:]
    dx   = ncid.variables['dx'][0,:]
    dz   = ncid.variables['dz'][:,0,:]
    lev  = ncid.variables['lev'][:]
    lon  = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    ncid.close()
    
    time10 = time[-780::]          #time[-129] is 2004-04-16 00:00:00
    v1 = v[-780::,:,:]
    tp50 = np.zeros(len(time10))
    #Compute transport for every v-point and last 15 years
    tp10 = np.zeros(np.shape(v1))
    for l in range (len(time10)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp10[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6
        tp50[l] = np.nansum(tp10[l,:,:])
    
    if i == 0:
        outfile = ('/home/users/jordi/tp1950.nc')    
        ncid = Dataset(outfile,'w')
        ncid.createDimension('time10',len(time10))    
        ncid.createVariable(totmod[i],'f8',('time10',))
        ncid.variables[totmod[i]][:]       = tp50
        ncid.close()
    else:    
        outfile = ('/home/users/jordi/tp1950.nc')    
        ncid = Dataset(outfile,'a')
        ncid.createVariable(totmod[i],'f8',('time10',))
        ncid.variables[totmod[i]][:]       = tp50
        ncid.close()

Excluding Bering Strait outflow

In [ ]:
totmod =  ['CAMS-CSM1-0','CAS-ESM2-0','CESM2-WACCM','CIESM','CMCC-CM2-SR5','CMCC-ESM2','CNRM-CM6-1','CNRM-ESM2-1','CanESM5','EC-Earth3','FIO-ESM-2-0',
            'HadGEM3-GC31-LL','HadGEM3-GC31-MM','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','UKESM1-0-LL']
ENS    =  ['r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2','r2i1p1f2','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f3',
           'r1i1p1f3','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f2']
EXP    = 'historical'

for i in range (len(totmod)):
    print('File ' + str(i+1))
    outfile = ('/home/users/jordi/section26data/' + totmod[i] + '_' + EXP + '_' + ENS[i] + '.nc')    
    ncid    = Dataset(outfile,'r')
    v       = ncid.variables['vo'][:,:,0,:]
    dx      = ncid.variables['dx'][0,:]
    dz      = ncid.variables['dz'][:,0,:]
    lev     = ncid.variables['lev'][:]
    lon     = ncid.variables['lon'][:,:]
    time    = ncid.variables['time'][:]
    ncid.close()
    
    outfile = ('/home/users/jordi/tp1950.nc')    
    ncid = Dataset(outfile,'r')    
    tpsum = ncid.variables[totmod[i]][:]
    ncid.close()
    
    dzdx = np.zeros(np.shape(dz))
    for j in range (len(dz)):
        for k in range (len(dx)):
            dzdx[j,k] = dz[j,k]*dx[k]
    
    #Exclude grid cells in MRI-ESM2
    if totmod[i] == 'MRI-ESM2-0':
        dzdx[-3,14] = 0
        dzdx[-4,31] = 0
        dzdx[-5,40] = 0
    
    dzdxsum  = np.nansum(np.nansum(dzdx,axis=0),axis=0)
    
    time10 = time[-780::]          #time[-129] is 2004-04-16 00:00:00
    v1 = v[-780::,:,:]
    #Compute transport for every v-point and last 15 years
    tp10_BS = np.zeros(np.shape(v1))
    for l in range (len(time10)):
        for j in range (len(dz)):
            for k in range (len(dx)):
                tp10_BS[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6 - (tpsum[l]*dzdx[j,k]/dzdxsum)   #Subtract Bering Strait outflow
                
    if i == 0:
        outfile = ('/home/users/jordi/tp1950_BS.nc')    
        ncid = Dataset(outfile,'w')
        ncid.createDimension('time10',len(time10))    
        ncid.createDimension(totmod[i] + '_lev',len(lev))    
        ncid.createDimension(totmod[i] + '_lon',len(np.squeeze(lon)))    
        ncid.createVariable(totmod[i],'f8',('time10',totmod[i] + '_lev',totmod[i] + '_lon',))
        ncid.variables[totmod[i]][:,:,:]       = tp10_BS
        ncid.close()
    else:    
        outfile = ('/home/users/jordi/tp1950_BS.nc')    
        ncid = Dataset(outfile,'a')
        ncid.createDimension(totmod[i] + '_lev',len(lev))    
        ncid.createDimension(totmod[i] + '_lon',len(np.squeeze(lon)))  
        ncid.createVariable(totmod[i],'f8',('time10',totmod[i] + '_lev',totmod[i] + '_lon',))
        ncid.variables[totmod[i]][:,:,:]       = tp10_BS
        ncid.close()

In [ ]:
totmod = 'CAMS-CSM1-0'
ENS    = 'r1i1p1f1'
EXP    = 'ssp585'

outfile = ('/home/users/jordi/section26data/' + totmod + '_' + EXP + '_' + ENS + '.nc')    
ncid = Dataset(outfile,'r')
v    = ncid.variables['vo'][:,:,0,:]
dx   = ncid.variables['dx'][0,:]
dz   = ncid.variables['dz'][:,0,:]
lev  = ncid.variables['lev'][:]
lon  = ncid.variables['lon'][:,:]
time    = ncid.variables['time'][:]
ncid.close()
    
time85 = time[0:85*12]
v1 = v[0:85*12,:,:]
#Compute transport for every v-point and last 15 years
tp85 = np.zeros(np.shape(v1))
for l in range (len(time85)):
    for j in range (len(dz)):
        for k in range (len(dx)):
            tp85[l,j,k] = v1[l,j,k]*dx[k]*dz[j,k]*1e-6
            
outfile = ('/home/users/jordi/section26data/' + totmod + '_' + EXP + '_' + ENS + '.nc')    
ncid = Dataset(outfile,'a')
ncid.createDimension('time85',len(time85))
ncid.createVariable('tp85','f8',('time85','lev','lon',))
ncid.variables['tp85'][:,:,:]       = tp85
ncid.close()